In [1]:
import pandas as pd
import nltk

In [2]:
def remove_stopwords(texto):
    frases_sem_stopwords = []
    for (frase, emocao) in texto:
        sem_stopwords = [palavra.lower() for palavra in frase.split() if palavra not in stopwordsNLTK]
        frases_sem_stopwords.append((sem_stopwords, emocao))
    
    return frases_sem_stopwords

def aplicar_stemmer(sem_stopwords):
    stemmer = nltk.stem.RSLPStemmer()
    frases = []
    
    for (frase, emocao) in sem_stopwords:  
        com_stemmer = []
        for palavra in frase:
            if palavra == 'doria':
                com_stemmer.append(palavra)
            else:
                com_stemmer.append(stemmer.stem(palavra))
        
        frases.append((com_stemmer, emocao))
        
    return frases

def lista_todas_as_palavras(frases):
    todas_palavras = []
    for (palavras, _) in frases:
        todas_palavras.extend(palavras)
        
    return todas_palavras

def busca_frequencia(palavras):
    return nltk.FreqDist(palavras)

def busca_palavras_unicas(frequencia):
    return frequencia.keys()

def extrator_palavras_treino(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_treino:
        caracteristicas['%s' %palavras] = (palavras in doc)
        
    return caracteristicas

def extrator_palavras_teste(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' %palavras] = (palavras in doc)
        
    return caracteristicas

def preditor(frase, classificador):
    testestmmer = []
    frase = frase.lower()
    stmmer = nltk.stem.RSLPStemmer()
    for palavra in frase.split():
        if palavra == 'doria':
            testestmmer.append(palavra)
        else:
            testestmmer.append(stmmer.stem(palavra))
    
    novo = extrator_palavras_treino(testestmmer)
    if classificador.classify(novo) == 0:
        print 'O comentário não é de um eleitor do Dória \n'
    elif classificador.classify(novo) == 1:
        print 'O comentário é de um eleitor do Dória \n'
        
    distribuicao = classificador.prob_classify(novo)
    for classe in distribuicao.samples():
        print "%d: %f" %(classe, distribuicao.prob(classe))

In [3]:
df_comentarios = pd.read_excel('comentarios4.xlsx') 

In [4]:
# alguns valores da Avaliação foram definidos como 'o'
# enquanto que deveria ser 0(zero)
df_comentarios.loc[df_comentarios['Avaliacao'] == u'o', 'Avaliacao'] = 0

In [5]:
df_comentarios.loc[df_comentarios['Avaliacao'] == u'/', 'Avaliacao'] = -1

In [6]:
df_comentarios.loc[df_comentarios['teste'] == u'x', 'teste'] = -1

In [7]:
df_comentarios.head(3)

,data,titulo,url,comentario,Avaliacao,teste
0,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,"Os mesmos de novo, com uma nova cara. Do jeito...",0,NaN
1,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,Doria nada mais é do que um cavalo de troia pa...,0,NaN
2,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,sou mais dar esse premio para o newton cardoso!!,NaN,NaN


In [8]:
# transformado NaN como -1
df_comentarios.fillna(-1, inplace=True)

In [9]:
df_comentarios.head(3)

,data,titulo,url,comentario,Avaliacao,teste
0,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,"Os mesmos de novo, com uma nova cara. Do jeito...",0,-1
1,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,Doria nada mais é do que um cavalo de troia pa...,0,-1
2,23/08/2017,João Doria tem título de cidadão honorário neg...,http://g1.globo.com/minas-gerais/noticia/joao-...,sou mais dar esse premio para o newton cardoso!!,-1,-1


In [10]:
df_comentarios_0_1 = df_comentarios[df_comentarios['Avaliacao'] != -1]

In [11]:
df_comentario_treino = df_comentarios_0_1[['comentario', 'Avaliacao']]

In [12]:
df_comentario_0_1_teste = df_comentarios[df_comentarios['teste'] != -1]

In [13]:
df_comentario_teste = df_comentario_0_1_teste[['comentario', 'teste']]

In [14]:
comentario_classe_treino = [tuple(valores) for valores in df_comentario_treino.values]

In [15]:
len(comentario_classe_treino)

587

In [16]:
comentario_classe_teste = [tuple(valores) for valores in df_comentario_teste.values]

In [17]:
len(comentario_classe_teste)

246

In [18]:
# StopWords em português
stopwordsNLTK = nltk.corpus.stopwords.words('portuguese')

In [19]:
# removendo StopWords dos comentários
sem_stopwords_treino = remove_stopwords(comentario_classe_treino)
sem_stopwords_teste = remove_stopwords(comentario_classe_teste)

In [20]:
# Estração do Radical (Stemmer)
radicais_treino = aplicar_stemmer(sem_stopwords_treino)
radicais_teste = aplicar_stemmer(sem_stopwords_teste)

In [21]:
# Criando lista de todas as palavras na minha base de comentários 
# que foram classifcados manualmente
todas_palavras_treino = lista_todas_as_palavras(radicais_treino)
todas_palavras_teste = lista_todas_as_palavras(radicais_teste)

In [22]:
# Extração de Palavras Únicas
frequencia_treino = busca_frequencia(todas_palavras_treino)
palavras_unicas_treino = busca_palavras_unicas(frequencia_treino)

frequencia_teste = busca_frequencia(todas_palavras_teste)
palavras_unicas_teste = busca_palavras_unicas(frequencia_teste)

In [23]:
len(palavras_unicas_treino)

3507

In [24]:
len(palavras_unicas_teste)

1459

In [25]:
# Extração de Palavras de Todas as Frases 
# classificadas manualemente para a formação da 
# matrix de probabilidades
basecompleta_treino = nltk.classify.apply_features(extrator_palavras_treino, radicais_treino)
basecompleta_teste =  nltk.classify.apply_features(extrator_palavras_teste, radicais_teste)

In [48]:
# treinando o nosso modelo Naive Bayes
classificador = nltk.NaiveBayesClassifier.train(basecompleta_treino)

In [38]:
classificador.labels()

[0, 1]

In [39]:
classificador.show_most_informative_features(5)

Most Informative Features
                 parabém = True                1 : 0      =     24.1 : 1.0
                   2018. = True                1 : 0      =      9.1 : 1.0
                   parab = True                1 : 0      =      7.8 : 1.0
                  melhor = True                1 : 0      =      7.3 : 1.0
                  negóci = True                0 : 1      =      6.0 : 1.0


In [46]:
preditor( u"Dória meu prefeito, meu presidente.", classificador)

O comentário é de um eleitor do Dória 

0: 0.022398
1: 0.977602


In [47]:
preditor('Acelera Sampa!!!! Doria o terror dos mortadelas', classificador)

O comentário é de um eleitor do Dória 

0: 0.008995
1: 0.991005


In [42]:
preditor(u'Esse Dória não me engana!', classificador)

O comentário não é de um eleitor do Dória 

0: 0.505969
1: 0.494031


In [49]:
preditor(u'Imaginar que esse almofadinha é um potencial presidente do país mostra bem que futuro nos espera', 
         classificador)

O comentário não é de um eleitor do Dória 

0: 0.989694
1: 0.010306


In [50]:
preditor(u'Globo já fazendo campanha antecipada pra esse novo Collor se lançar a presidência.', classificador)

O comentário é de um eleitor do Dória 

0: 0.395375
1: 0.604625


In [35]:
comentario_teste6 = u'esse cara não me parece uma pessoa honesta, coxinha'

In [36]:
preditor(comentario_teste6, classificador)

O comentário não é de um eleitor do Dória 

0: 0.843067
1: 0.156933


# Testando acurácia do modelo

In [39]:
nltk.classify.accuracy(classificador, basecompleta_teste)

0.7398373983739838

# Matriz de Confusão do Modelo

In [40]:
esperado = []
previsto = []
for comentario, classe in basecompleta_teste:
    resultado = classificador.classify(comentario)
    previsto.append(resultado)
    esperado.append(classe)

In [41]:
from nltk.metrics import ConfusionMatrix

matriz = ConfusionMatrix(esperado, previsto)
print matriz

  |   0   1 |
--+---------+
0 | <80> 43 |
1 |  21<102>|
--+---------+
(row = reference; col = test)



# Criando nuvem de palavras

In [42]:
palavras_eleitores_doria = [comentario for comentario, classe in comentario_classe_treino
                           if classe == 1]

palavras_eleitores_doria += [comentario for comentario, classe in comentario_classe_teste
                           if classe == 1]

palavras_eleitores_doria = ''.join(palavras_eleitores_doria)

In [43]:
print palavras_eleitores_doria

coitado do marginal que jogava ovo nos outros....Doria ja falou que vai acabar com a estabilidade no serviço público e quem quiser se manter no cargo vai ter que trabalhar e mostrar serviço, só isso já basta pra deixar a esquerda furiosa.PELO JEITO TEMOS MUITOS MORTADELAS QUE ESTAO COM MEDO DA MAMATA ACABAR NEH???? DÓRIA SERÁ O PRÓXIMO PRESIDENTE DO BRASIL E ESSA CORJA DE INTOLERANTES IRÁ ACABAR......AGUARDEM.....ACELERA DÓRIA!!Este estado é o pior do Brasil, pobre em cultura, pobre em td. tem que se ferrar. Sou a favor daquele reitor antigo que diz:Oo pai dele foi deputado federal pela Bahia, ok, então está explicado e mais que justo o motivo do titulo de cidadão Soteropolitano, que que é um descendente! O povinho da esquerda protestando é algo muito ridículo.já pensou se SP deportasse todos os baianos de volta para seu estado de origem ?...Estes nordestinos tem que agradecer o dinheiro de SP que vai pra eles!Nordeste tem um povo atrasado igual no Oriente Médio, se acordassem pra real

In [44]:
palavras_nao_eleitores_doria = [comentario for comentario, classe in comentario_classe_treino
                           if classe == 0]

palavras_nao_eleitores_doria += [comentario for comentario, classe in comentario_classe_teste
                           if classe == 0]

palavras_nao_eleitores_doria = ''.join(palavras_nao_eleitores_doria)

In [45]:
print palavras_nao_eleitores_doria

Os mesmos de novo, com uma nova cara. Do jeito que a população são fáceis de serem manipuladas, tá arriscado mesmo esse politiquinho de pulôver ser eleito presidente.Doria nada mais é do que um cavalo de troia para pegar o voto dos desavisados e manter os corrompidos no poder! Eu não entro nessa arapuca não! Sou B0LS0NAR02018!ESSE IDIOTA DESSE CORREIA...DEVERIA TER VERGONHA NA CARA...EM VEZ DE PROMOVER ALGUEM..QUE NADA FEZ PELO BRASIL ATE AGORA..PRINCIPALMENTE POR SAO PAULO...QUE JOGADO AS MINGUAS..É muito sem sentido esses títulos de cidadania pra quem nunca fez nada pela região. O que esse cara fez por Minas ou pela Bahia onde levou aquelas ovadas....Apenas promoção pessoal, campanha política descarada. Vergonha!Esse play mal esquentou a cadeira de prefeito foi picado pela mosca azul e já sonha com o planalto, quer ser o novo Emmanuel Macron das Américas.....É muita pretensão!Qualquer cidadão mineiro trabalhador e pagador de impostos merece mais esse prêmio do que ele.Concordo! Manda

In [74]:
f = open('palavras_eleitores_doria.txt', 'w')
f.write(palavras_eleitores_doria.encode('utf-8'))
f.close()

In [75]:
f = open('palavras_nao_eleitores_doria.txt', 'w')
f.write(palavras_nao_eleitores_doria.encode('utf-8'))
f.close()

In [76]:
doria = open('palavras_eleitores_doria.txt', 'r')

In [78]:
teste = doria.readlines()

In [81]:
palavras = ''
for i in teste:
    palavras += i
    
print palavras

coitado do marginal que jogava ovo nos outros....Doria ja falou que vai acabar com a estabilidade no serviço público e quem quiser se manter no cargo vai ter que trabalhar e mostrar serviço, só isso já basta pra deixar a esquerda furiosa.PELO JEITO TEMOS MUITOS MORTADELAS QUE ESTAO COM MEDO DA MAMATA ACABAR NEH???? DÓRIA SERÁ O PRÓXIMO PRESIDENTE DO BRASIL E ESSA CORJA DE INTOLERANTES IRÁ ACABAR......AGUARDEM.....ACELERA DÓRIA!!Este estado é o pior do Brasil, pobre em cultura, pobre em td. tem que se ferrar. Sou a favor daquele reitor antigo que diz:Oo pai dele foi deputado federal pela Bahia, ok, então está explicado e mais que justo o motivo do titulo de cidadão Soteropolitano, que que é um descendente! O povinho da esquerda protestando é algo muito ridículo.já pensou se SP deportasse todos os baianos de volta para seu estado de origem ?...Estes nordestinos tem que agradecer o dinheiro de SP que vai pra eles!Nordeste tem um povo atrasado igual no Oriente Médio, se acordassem pra real